# Introduction to Text Classification II

-----

In this notebook, we explore more advanced machine learning techniques with text data. First, we further explore the text classification we did in the previous lesson to see how we can improve. Next, we introduce the concept of n-grams, which are combinations of one or more tokens. For example, bigrams are combinations of two tokens, while trigrams are combinations of three tokens. Finally, we introdce sentiment analysis with a new text data set.

-----

## Table of Contents

[Explore Classifiers](#Explore-Classifiers)

[N-Grams](#N-Grams)

[N-Gram Classification](#N-Gram-Classification)

[Sentiment Analysis](#Sentiment-Analysis)

[Stemming](#Stemming)

-----

Before proceeding with the rest of this notebook, we first include the notebook setup code and we define our _home_ directory.

-----

In [1]:
# Set up Notebook
%matplotlib inline

# Standard imports
import numpy as np

# We do this to ignore several specific warnings
import warnings
warnings.filterwarnings("ignore")


## Explore Classifiers

Let's explore the classifiers we used in the previous lesson. We can get top features(words, tokens) used by a classifier to predict each class. By examining the top words of each class, we get better understanding of the data set and the classifier. We hope this will help us improve the classification.

We first repeat the text classification on the twenty newsgroup dataset with MultinomialNB classifier. In the next Code cell, we load the data and create DTM with `TfidfVectorizer`, then train the `MultinomialNB` model. 

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.datasets import fetch_20newsgroups

# To learn more about these data, either browse the relevant 
# scikit-learn documentation, or enter help(fetch_20newsgroups) in a Code cell

# Get training text set
train = fetch_20newsgroups(data_home='.', subset='train')
# Get testing text set
test = fetch_20newsgroups(data_home='.', subset='test')

# Create DTM
tf_cv = TfidfVectorizer(stop_words='english')
train_dtm_tf = tf_cv.fit_transform(train['data'])
test_dtm_tf = tf_cv.transform(test['data'])

nb = MultinomialNB()
nb = nb.fit(train_dtm_tf, train['target'])

predicted = nb.predict(test_dtm_tf)
scr = 100.0 * nb.score(test_dtm_tf, test['target'])
print(f'NB (TF-IDF with Stop Words) prediction accuracy = {scr:5.1f}%')

NB (TF-IDF with Stop Words) prediction accuracy =  81.7%


-----

We can use the `coef_` attribute of a linear classifier to identify the top features (words, tokens) for each category. (A linear classifier makes a classification decision based on the value of a linear combination of the characteristics or features. i.e., MultinomialNB, LinearSVC, LogisticRegressoin). `coef_` holds coefficients of all the features for each class. Since in text analysis, the features are tokens (words) in the whole training text set. A larger coefficient means more impact of the feature on predicting the class.

In the following Code cell we use `coef_` to find out top 5 features (words, tokens) used to predicting each class. I'll explain the code a bit:

- First get all features from the vectorizer and convert the return value to a numpy array.
- Iterate through all classes, in our case, the 20 newsgroups.
- For each class, sort the classifiers' `coef_` attribute. For example, `nb.coef_[0]` is a list of coefficients for the first class, or _alt.atheism_ in our case. `np.argsort` is a numpy sort function that sorts a numpy array, but return a numpy array with indexes of the sorted array. For example, `np.argsort(np.array([2, 1, 3]))` returns a numpy array `[1, 0, 2]`. Because in the original array, the item with index 1 is the smallest, the item with index 0 is the second smallest and the item with index 2 is the largest.
- Once we get an index of sorted coefficients, keep the last 5 indexes as top_word_index since those indexes are corresponding to the top 5 largest coefficients.
- Use the top_word_index which is an array of indexes to find the top feature names(words).
- Reverse the list of words so that the most important one is at the first.

-----

In [3]:
# Display top 5 important words
all_words = np.array(tf_cv.get_feature_names())

for idx, target in enumerate(train['target_names']):
    top_word_index = np.argsort(nb.coef_[idx])[-5:]
    tn_lst = [word for word in all_words[top_word_index]]
    tn_lst.reverse()

    print(f'\n{target}:')
    print(tn_lst)


alt.atheism:
['keith', 'edu', 'god', 'caltech', 'atheists']

comp.graphics:
['graphics', 'edu', 'image', '3d', 'files']

comp.os.ms-windows.misc:
['windows', 'edu', 'file', 'dos', 'files']

comp.sys.ibm.pc.hardware:
['scsi', 'drive', 'ide', 'card', 'edu']

comp.sys.mac.hardware:
['mac', 'apple', 'edu', 'drive', 'quadra']

comp.windows.x:
['window', 'motif', 'mit', 'server', 'com']

misc.forsale:
['sale', 'edu', '00', 'offer', 'shipping']

rec.autos:
['car', 'com', 'cars', 'edu', 'engine']

rec.motorcycles:
['bike', 'com', 'dod', 'edu', 'ride']

rec.sport.baseball:
['edu', 'baseball', 'year', 'team', 'game']

rec.sport.hockey:
['hockey', 'team', 'game', 'ca', 'edu']

sci.crypt:
['key', 'clipper', 'encryption', 'chip', 'com']

sci.electronics:
['edu', 'com', 'use', 'lines', 'subject']

sci.med:
['pitt', 'edu', 'geb', 'banks', 'gordon']

sci.space:
['space', 'nasa', 'edu', 'henry', 'moon']

soc.religion.christian:
['god', 'jesus', 'christians', 'church', 'edu']

talk.politics.guns:
['gun

---
From the top words that are used to identify each class(label), we can see that "edu" and "com" are in the top 5 words of many classes. Those words are from email addresses which every message has. They don't really have much predicting power. The fact that they are in top 5 of so many classes proves this. In the previous lesson we mentioned that we hope that TF-IDF can help mitigate the problem but seems it doesn't. We can solve this problem by defining our own stop words and add "com" and "edu" to the stop words to filter them out manually. We'll also add "re" which is in many message headers to the stop words. We demonstrate this approach in the following Code cell.

The change doesn't improve the classification accuracy. This shows that not all improvements work as desired. But with the customized stop words, we do get more meaningful top words. This can be very valuable information sometimes.

In [1]:
#get current stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

#add com, edu and re to stop words
stop_words.extend(['com', 'edu', 're'])

# Create DTM, use custom defined stop words
tf_cv = TfidfVectorizer(stop_words=stop_words)
train_dtm_tf = tf_cv.fit_transform(train['data'])
test_dtm_tf = tf_cv.transform(test['data'])

nb = MultinomialNB()
nb = nb.fit(train_dtm_tf, train['target'])

predicted = nb.predict(test_dtm_tf)
scr = 100.0 * nb.score(test_dtm_tf, test['target'])
print(f'NB (TF-IDF with Stop Words) prediction accuracy = {scr:5.1f}%')

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/home/jovyan/nltk_data'
    - '/opt/conda/nltk_data'
    - '/opt/conda/share/nltk_data'
    - '/opt/conda/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
# Display top 5 important words
all_words = np.array(tf_cv.get_feature_names())

for idx, target in enumerate(train['target_names']):
    top_word_index = np.argsort(nb.coef_[idx])[-5:]
    tn_lst = [word for word in all_words[top_word_index]]
    tn_lst.reverse()

    print(f'\n{target}:')
    print(tn_lst)

---
Next, we apply LogisticRegression on the DTM which is created with customized stop words, but the classification accuracy doesn't improve again. Then we print out the top 5 words from each class. We can see that `LogisticRegerssion` gives different top words to that of `MultinomialNB`. This is because the different models use different features for prediction. There are still quite a few overlaps of words in the top words of each class.

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1000)

lr = lr.fit(train_dtm_tf, train['target'])
predicted = lr.predict(test_dtm_tf)

scr = 100.0 * lr.score(test_dtm_tf, test['target'])
print(f'LR (TF-IDF with Stop Words) prediction accuracy = {scr:5.1f}%')

In [ ]:
for idx, target in enumerate(train['target_names']):
    top_word_index = np.argsort(lr.coef_[idx])[-5:]
    tn_lst = [word for word in all_words[top_word_index]]
    tn_lst.reverse()

    print(f'\n{target}:')
    print(tn_lst)

-----

<font color='red' size = '5'> Student Exercise </font>

In the preceding cells, we print out top words used in predicting each class for MultinomialNB and LogisticRegression. In the following Code cell, try to display the top 5 words used by LinearSVC.

-----

[[Back to TOC]](#Table-of-Contents)

## N-Grams

Formally, a [_n-gram_][ngd] is a contiguous sequence of **n** items from a parent sequence of items, such as characters or words in a text document. In general, we will focus solely on words in a document. Thus, our initial approach has simply been to look at unigrams or single words in a document when building a classification model. However, sometimes the combination of words can be more descriptive, for example, _unbelievably bad_ is generally viewed as a more powerful description than just _bad_. As a result, the concept of an _n-gram_ was created, where collections of words can be treated as features. In fact Google allows a user to search for [specific n-gram][gnv] combinations in books that they have scanned.

While this clearly can improve classification power, it also increases computational requirements. This is a result of the rise in the number of possible features. While this is not a problem for small vocabularies, for larger vocabularies (and corresponding documents) the number of possible features can quickly become very large. Thus, many text mining applications will make use of Hadoop or Spark clusters to leverage the inherent parallelism in these tasks.

To demonstrate using n-grams, we first demonstrate the concept on a single sentence.

-----
[gnv]: https://books.google.com/ngrams
[ngd]: https://en.wikipedia.org/wiki/N-gram

In [ ]:
my_text = 'This course introduces many concepts in data science.'

# Tokenize sentance
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1,3))

# Analyze sentance
tk_func = cv.build_analyzer()

# Display n-grams
print(tk_func(my_text))

-----
`my_text` has 8 words. When we set `ngram_range=(1,3)` which means create tokens for 1, 2 and 3 consecutive words, there are 21 tokens now.

We can create a new document-term matrix based on this sentence, then use this matrix (or vector since it is only one row), to provide a representation space for new sentences. In the following Code cell, we tokenize our original sentence, and sort the resulting vocabulary (i.e., n-grams) with their ranking (or order). Next, we create a second, simple sentence and compute the indices into the original DTM for the n-grams in the new sentence. The result is displayed as a bit vector, where `1` means the corresponding n-gram is in the new sentence, and `0` means it is not.


-----

In [ ]:
# Tokenize sentence
cv = cv.fit([my_text])

# Sort tokens
import operator
my_voc = sorted(cv.vocabulary_.items(), key=operator.itemgetter(1))

# Display token mapping
print('Token mapping:')
print(40*'-')

for tokens, rank in my_voc:
    print(rank, tokens)

# Display new sentence
print(40*'-')
out_list = ['This course is data science!']

# Transform new sentence to original sentence DTM
xsm = cv.transform(out_list)
print(out_list)

# Display count vector indices for new sentance tokens
print(40*'-')
print(xsm.todense())

-----

<font color='red' size = '5'> Student Exercise </font>

In the preceding cells, we used `CountVectorizer` to create n-gram tokens. Now that you have run the notebook, go back and make the following changes to see how the results change.

1. Change the `CountVectorizer` to use stop words. How does this change the tokens and mappings? 
2. Try changing the ngram range to different values (e.g., `ngram_range=(1,4)`). Notice how the number of tokens quickly increase.

-----

[[Back to TOC]](#Table-of-Contents)

## N-Gram Classification

Having n-grams often offers improved classification, since word or token combinations often include more information than single words or tokens. For example, _University Illinois_ means more than just _University_ and _Illinois_. We can build on our previous simple text classification to now develop a more complete code example that builds a feature vector containing both single words and n-grams from the documents. We use this new sparse matrix to classify the documents by using our simple Naive Bayes classifier. However, we actually get a slightly worse accuracy score. This shows again that not all the optimization attempts will improve the model performance.

-----

In [ ]:
# Create DTM
tf_cv = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
train_dtm_tf = tf_cv.fit_transform(train['data'])
test_dtm_tf = tf_cv.transform(test['data'])

nb = MultinomialNB()
nb = nb.fit(train_dtm_tf, train['target'])

predicted = nb.predict(test_dtm_tf)
scr = 100.0 * nb.score(test_dtm_tf, test['target'])
print(f'NB (TF-IDF with ngram_range 1-2) prediction accuracy = {scr:5.1f}%')

-----

<font color='red' size = '5'> Student Exercise </font>

In the preceding cells, we set ngram_range to (1,2) and apply MultinomialNB. In the next Code cell, try applying other classifiers like LinearSRC or LogisticRegression with ngram_range (1,2) to see how the results change.



-----

[[Back to TOC]](#Table-of-Contents)


## Sentiment Analysis

Sentiment Analysis aims to identify and extract emotion or opinion from text. It is a useful technology that businesses can apply in social media, customer reviews, and customer support.

In this section, we turn to a classification problem where the goal is to classify based on sentiment, negative or positive. We will approach this as a simple classification problem. 

We first load our data. We will use the movie reviews data built-in in the `nltk.corpus` module. The movie_reviews corpus contains 2000 movie reviews with sentiment polarity classification. Out of the 2000 reviews, the first 1000 are negative reviews, and next 1000 are positive reviews.

In the following Code cell, we first load the movie reviews corpus from ntlk. Each review has a sentiment label, either _'neg'_ or _'pos'_ (stands for negative and positive respectively). The reviews are stored as a list of words. We combine the list of words to generate the text reviews, which will be our text data. We then create classification labels by mapping _'neg'_ to 0 and _'pos'_ to 1. Finally, we split the data set to training and testing date set. We print out the first review and its label in the training text set to show what a review looks like.

**Note:** Don't worry if you have trouble understanding the code to load the movie reviews from nltk. Just focus on the final text data set.

-----

In [ ]:
import random
from sklearn.model_selection import train_test_split
from nltk.corpus import movie_reviews
#load movie reviews, each review is a list of words
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

print ("Number of Reviews:", len(documents))
#shuffle reviews to mix negative and positive reviews
#set random seed for reproducibility
random.seed(23)
random.shuffle(documents)

#list to store all review text
text_data = []
#label
label = []
for i in range(len(documents)):
    #join list of words to create a review and add to text_data
    text_data.append(' '.join(documents[i][0]))
    #map neg to 0, pos to 1 and add to label
    label.append(0 if documents[i][1]=='neg' else 1)

print("Number of Negative Reviews:", label.count(0))
print("Number of Positive Reviews:", label.count(1))    

#split to train and text
mvr_train, mvr_test, y_train, y_test = train_test_split(text_data, label, test_size=0.25, random_state=23)

#print one example review in the training text set
print("Sample Review:")
print('Label:', y_train[0])
print(mvr_train[0])

---
Next we create bag of words from training text set with `TfidfVectorizer`, transform both training and testing set to the document term matrix, then apply `MultinomaiNB` classifier. We print out the score, the classification report and plot the confusion matrix.

---

In [ ]:
# Naive Bayes pipeline to classify
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

tf_cv = TfidfVectorizer(stop_words='english')
train_dtm_tf = tf_cv.fit_transform(mvr_train)
test_dtm_tf = tf_cv.transform(mvr_test)

# Fit model, predict, and display results
nb = MultinomialNB()
nb = nb.fit(train_dtm_tf, y_train)
y_pred = nb.predict(test_dtm_tf)
scr = 100.0 * nb.score(test_dtm_tf, y_test)
print(f'NB (TF-IDF with stop words) prediction accuracy = {scr:5.1f}%')
print(metrics.classification_report(y_test, y_pred, target_names = ['Negative', 'Positive']))

In [ ]:
from helper_code import mlplots as ml

#fig, ax = plt.subplots(figsize=(13, 10))
# Call confusion matrix plotting routine
ml.confusion(y_test, y_pred, ['Negative', 'Positive'], 'Naive Bayes Model')

---
Next we display top words in predicting positive reviews. Note that since there are only two classes, either 0 or 1, there's only one list of coefficients for the positive class. Because the model only needs to classify positive reviews and the rest are negative reviews. If you want to find out top words for predicting negative reviews, you may reverse the label, map _neg_ to 1 and *pos* to 0 and repeat following code.

We print out the top 20 words for predicting positive movie reviews. There are good, great, best and love which make sense, but there are also many words like film, movie, story, and really, etc. that don't make much sense. We can try to add words like film and movie to the stop words since those words are likely to appear in both positive and negative reviews. Or we can try another algorithm to see if we get better result.

In [ ]:
all_words = np.array(tf_cv.get_feature_names())

top_word_index = np.argsort(nb.coef_[0])[-20:]
tn_lst = [word for word in all_words[top_word_index]]
tn_lst.reverse()

print(f'\nTop 20 Words of Positive Reviews:')
print(tn_lst)

---
We now try `LogisticRegression` with `C` set to `1000`. This model gives better accuracy and a much better list of top words for predicting positive reviews.

---

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=1000)

lr = lr.fit(train_dtm_tf, y_train)
predicted = lr.predict(test_dtm_tf)

scr = 100.0 * lr.score(test_dtm_tf, y_test)
print(f'LR (TF-IDF with Stop Words) prediction accuracy = {scr:5.1f}%')

In [ ]:
top_word_index = np.argsort(lr.coef_[0])[-20:]
tn_lst = [word for word in all_words[top_word_index]]
tn_lst.reverse()

print(f'\nTop 20 Words of Positive Reviews:')
print(tn_lst)

---
Next we reverse the label value, train the model again and get the top words for predicting negative reviews. Most words in the list make a lot of sense.

---

In [ ]:
#reverse label value so that negative reviews have label 1
y_train_reverse = [0 if y==1 else 1 for y in y_train]
lr = lr.fit(train_dtm_tf, y_train_reverse)

top_word_index = np.argsort(lr.coef_[0])[-20:]
tn_lst = [word for word in all_words[top_word_index]]
tn_lst.reverse()

print(f'\nTop 20 words of Negative Reviews:')
print(tn_lst)

-----

<font color='red' size = '5'> Student Exercise </font>

In the previous Code cells, we classified movie reviews with default English stop words. Try adding common words in movie reviews like movie and film to the stop words. What impact does this change have on each model?

-----

[[Back to TOC]](#Table-of-Contents)

## Stemming

In the previous lesson, we introduced the concept of stemming. In the next Code cell, we demonstrate how to apply stemming in text classification. We will use `PorterStemmer` in the `nltk` module for stemming.

We first define a function `tokenize`. The function has one argument `text` which is the text to be tokenized. The function uses `nltk.word_tokenize` function to tokenize `text` then apply `PorterStemmer` to stem the tokens. We then  set `tokenizer` argument in `CounterVectorizer` or `TfidfVectorizer` with this custom `tokenize` function and use the new vectorizer to create bag of words.

The following code takes longer to finish due to stemming, but it does give a better classification accuracy.

-----
[ws]: https://en.wikipedia.org/wiki/Stemming

In [ ]:
import string
import nltk
from nltk.stem.porter import PorterStemmer

# Define function to tokenize text and apply stemmer
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token for token in tokens if token not in string.punctuation]

    stemmer = PorterStemmer()
    stems = map(stemmer.stem, tokens)
    return stems

# use custom tokenize when creating vectorizer
tf_cv = TfidfVectorizer(tokenizer=tokenize)
train_dtm_tf = tf_cv.fit_transform(mvr_train)
test_dtm_tf = tf_cv.transform(mvr_test)

lr = LogisticRegression(C=1000)

lr = lr.fit(train_dtm_tf, y_train)
predicted = lr.predict(test_dtm_tf)

scr = 100.0 * lr.score(test_dtm_tf, y_test)
print(f'LR (TF-IDF with Stemming) prediction accuracy = {scr:5.1f}%')


## Ancillary Information

The following links are to additional documentation that you might find helpful in learning this material. Reading these web-accessible documents is completely optional.

1. Wikipedia articles on [n-grams][wng], [Stemming][wst], and [Lemmatization][wl]
1. Google [n-gram viewer][gnv]
1. Alternative [n-gram viewer][anv]
1. Blog on Sentiment Analysis with NLTK, [Part III][bsa3] and [Part IV][bsa4]
1. An online [Stemming Demo][std] using NLTK
1. A [treatise on Snowball][tsb] discussing, in depth, the process of stemming.

-----

[wst]: https://en.wikipedia.org/wiki/Stemming
[wl]: https://en.wikipedia.org/wiki/Lemmatisation
[wtc]: https://en.wikipedia.org/wiki/Document_clustering

[tsb]: http://snowball.tartarus.org/texts/introduction.html
[std]: http://text-processing.com/demo/stem/

[wng]: https://en.wikipedia.org/wiki/N-gram

[gnv]: https://books.google.com/ngrams
[anv]: http://xkcd.culturomics.org

[bsa3]: http://streamhacker.com/2010/05/24/text-classification-sentiment-analysis-stopwords-collocations/
[bsa4]: http://streamhacker.com/2010/05/24/text-classification-sentiment-analysis-stopwords-collocations/

[msdr]: http://research.microsoft.com/pubs/150728/FnT_dimensionReduction.pdf
[lle]: http://science.sciencemag.org/content/290/5500/2323.abstract
[ica]: http://www.cs.rutgers.edu/~mlittman/topics/dimred02/kolenda99independent.pdf1. 

**&copy; 2019: Gies College of Business at the University of Illinois.**

This notebook is released under the [Creative Commons license CC BY-NC-SA 4.0][ll]. Any reproduction, adaptation, distribution, dissemination or making available of this notebook for commercial use is not allowed unless authorized in writing by the copyright holder.

[ll]: https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode